In [ ]:
import os
import shutil
import random
import json
import requests
import re

In [ ]:
USE_DRIVE = True
if USE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    project_path = "drive/MyDrive/EFREI_CAMP/"
else:
    project_path  = "../"
out_path  = os.path.join(project_path, "training/data/unlabeled")

In [ ]:
out_data_path = os.path.join(out_path,"to_label")
if os.path.isdir(out_data_path):
    shutil.rmtree(out_data_path)
    os.makedirs(out_data_path)
else:
    os.makedirs(out_data_path)

## BUILD DATASET FOR LABELLING

In [ ]:
ent_min_tr = list(range(1, 70, 2))
ent_scd_tr = list(range(2, 90, 2))
ent_hr_tr = list(range(2, 30, 2))

ent_min_tr_plus = ["et quart", "moins quart", "et demi"]
ent_scd_tr_plus = list(range(1,60,1))

minutes_tr_rg = list(zip([" ".join([str(el), random.choice(["minute", "minutes"])]) for el in ent_min_tr],
                         [str(el) for el in ent_min_tr],
                         ["duration_wt_min" for el in ent_min_tr]))
seconds_tr_rg = list(zip([" ".join([str(el), random.choice(["seconde", "secondes"])]) for el in ent_scd_tr],
                         [str(el) for el in ent_scd_tr],
                         ["duration_wt_sd" for el in ent_scd_tr]))
hours_tr_rg = list(zip([" ".join([str(el), random.choice(["heure", "heures"])]) for el in ent_hr_tr],
                         [str(el) for el in ent_hr_tr],
                         ["duration_wt_hr" for el in ent_hr_tr]))

minutes_tr_plus_rg = list(zip([" ".join([str(el)]) for el in ent_min_tr_plus],
                         [str(el) for el in ent_min_tr_plus],
                         ["duration_wt_min" for el in ent_min_tr_plus]))

seconds_tr_plus_rg = list(zip([" ".join([str(el)]) for el in ent_scd_tr_plus],
                         [str(el) for el in ent_scd_tr_plus],
                         ["duration_wt_sd" for el in ent_scd_tr_plus]))
minutes_tr_plus_rg = [list(el) for el in minutes_tr_plus_rg]
seconds_tr_plus_rg = [list(el) for el in seconds_tr_plus_rg]

E_range = ["entre chaque série", "entre séries"]

ent_min_br = list(range(1, 300, 2))
ent_scd_br = list(range(15, 300, 3))
ent_hr_br = list(range(1, 20, 2))

minutes_br_rg = list(zip([" ".join([str(el), random.choice(["minute", "minutes"])]) for el in ent_min_br for el2 in E_range],
                         [str(el) for el in ent_min_br for _ in E_range],
                        ["duration_br_min" for el in ent_min_br for _ in E_range]))
seconds_br_rg = list(zip([" ".join([str(el), random.choice(["seconde", "secondes"])]) for el in ent_scd_br for el2 in E_range],
                         [str(el) for el in ent_scd_br for _ in E_range],
                        ["duration_br_sd" for el in ent_scd_br for _ in E_range]))
hours_br_rg = list(zip([" ".join([str(el), random.choice(["heure", "heures"])]) for el in ent_hr_br for el2 in E_range],
                       [str(el) for el in ent_hr_br for _ in E_range],
                      ["duration_br_hr" for el in ent_hr_br for _ in E_range]))

minutes_br_plus_rg = list(zip([" ".join([str(el)]) for el in ent_min_tr_plus],
                         [str(el) for el in ent_min_tr_plus],
                         ["duration_br_min" for el in ent_min_tr_plus]))

seconds_br_plus_rg = list(zip([" ".join([str(el)]) for el in ent_scd_tr_plus],
                         [str(el) for el in ent_scd_tr_plus],
                         ["duration_br_sd" for el in ent_scd_tr_plus]))
minutes_br_plus_rg = [list(el) for el in minutes_br_plus_rg]
seconds_br_plus_rg = [list(el) for el in seconds_br_plus_rg]

durations_tr = minutes_tr_rg + seconds_tr_rg + hours_tr_rg
durations_br = hours_br_rg + minutes_br_rg + seconds_br_rg
durations_tr = [list(el) for el in durations_tr]
durations_br = [list(el) for el in durations_br]

rounds_range = list(zip([str(el) for el in list(range(1,50,2))],
                       ["nb_rounds" for el in list(range(1,50,2))]))
rounds_range = [list(el) for el in rounds_range]

random.shuffle(rounds_range)
random.shuffle(durations_br)
random.shuffle(durations_tr)

B_range = ["crée", "prépare", "initie", "commence", " ", "débute", "génère", "lance"]
SET_range = ["série", "set", "séquence", "cycle"]

In [ ]:
Dataset_1 = []

for duration_br in durations_br:
    for ent_rnd in rounds_range:
        for idx,duration_tr in enumerate(durations_tr):
            Beg = random.choice([random.choice(B_range), random.choice(B_range) + "s"])
            Beg = Beg.rstrip(" s ")
            SET = random.choice([random.choice(SET_range), random.choice(SET_range) + "s"])
            
            random_minutes_tr_plus_rg = random.choice(minutes_tr_plus_rg)
            random_seconds_tr_plus_rg = random.choice(seconds_tr_plus_rg)
            random_minutes_tr_plus_rg_random = random.choice([random_minutes_tr_plus_rg[0],""])
            random_seconds_tr_plus_rg_random = random.choice([random_seconds_tr_plus_rg[0],""])
            
            random_minutes_br_plus_rg = random.choice(minutes_br_plus_rg)
            random_seconds_br_plus_rg = random.choice(seconds_br_plus_rg)
            random_minutes_br_plus_rg_random = random.choice([random_minutes_br_plus_rg[0],""])
            random_seconds_br_plus_rg_random = random.choice([random_seconds_br_plus_rg[0],""])
            E_range_random = random.choice(E_range)
            
            if duration_tr[-1] == 'duration_wt_min' and random_seconds_tr_plus_rg_random: 
                TEXT = " ".join([Beg, ent_rnd[0], SET, "de",
                             duration_tr[0], random_seconds_tr_plus_rg_random])
            elif duration_tr[-1] == 'duration_wt_hr' and random_minutes_tr_plus_rg_random: 
                TEXT = " ".join([Beg, ent_rnd[0], SET, "de",
                             duration_tr[0], random_minutes_tr_plus_rg_random])
            else:
                TEXT = " ".join([Beg, ent_rnd[0], SET, "de",
                             duration_tr[0]])
            
            if duration_br[-1] == 'duration_br_min' and random_seconds_br_plus_rg_random: 
                TEXT = " ".join([TEXT, duration_br[1],
                                 ''.join([i for i in duration_br[0] if not i.isdigit()]).strip(),#get only str from duration_br[0]
                                 random_seconds_br_plus_rg_random, E_range_random])
            elif duration_br[-1] == 'duration_br_hr' and random_minutes_br_plus_rg_random: 
                TEXT = " ".join([TEXT, duration_br[1],
                                 ''.join([i for i in duration_br[0] if not i.isdigit()]).strip(),
                                 random_minutes_br_plus_rg_random, E_range_random])
            else:
                TEXT = " ".join([TEXT,
                                 random.choice([" ".join([duration_br[0], E_range_random]),""])])

            TEXT_cln = re.sub(r'\s+', ' ', TEXT)
            Dataset_1.append(TEXT_cln)
                    
print(len(Dataset_1))

In [ ]:
# Select data size that will be used for annotation
DATA_SIZE = 5000 #EDIT
random.shuffle(Dataset_1)
to_label = random.sample(Dataset_1, DATA_SIZE)
random.shuffle(to_label)
print(to_label[:10])

In [ ]:
# Save sentences in .txt format
for idx_sent,sentence in enumerate(to_label):
    file_path = os.path.join(out_data_path, "{}.txt".format(idx_sent))
    with open(file_path, "w") as text_file:
        text_file.write(sentence)

# Create tasks.json (data that will be sent to LabelStudio)
tasks = []
for filename in os.listdir(out_data_path):

    with open(os.path.join(out_data_path,filename)) as f:
        text = f.read()

    tasks.append({
            'data': {'text': text,
                    'filename': filename}
        })
with open(os.path.join(out_path,'tasks.json'), mode='w') as f:
    json.dump(tasks, f, indent=4)

print(os.listdir(out_path))

#### Expected output : `['to_label', 'tasks.json']`

## SEND DATASET TO LABELSTUDIO FOR ANNOTATION
### Edit `Notebooks/labelstudio_auth.json` before running the cell below

#### Expected output `<Response [201]>`

In [ ]:
with open(os.path.join(project_path, 'Notebooks/labelstudio_auth.json')) as json_file:
    labelstudio_auth = json.load(json_file)
    print(labelstudio_auth)

TOKEN = labelstudio_auth["TOKEN"]
URL = labelstudio_auth["URL"]
PROJECT_ID = labelstudio_auth["PROJECT_ID"]
local_file = os.path.join(out_path,'tasks.json')
headers = {
    'Authorization': 'Token {}'.format(TOKEN),
}
files = {
    'file': (local_file, open(local_file, 'rb')),
}

response = requests.post(os.path.join(URL,'api/projects/{}/import'.format(PROJECT_ID)),
                         headers=headers,
                         files=files)
print(response)